In [1]:
def locate_next(words, rules, level=1):
    try:
        lista = words.split(' ')
        next_level = {}
        url = ""
        for key, value in rules.items():
            if key == lista[0]:
                # print('key found =', lista[0])
                if isinstance(value, dict):
                    for k, v in value.items():
                        next_level[k] = v
                else:
                    url = value

        if url:
            return url, level
        else:
            if len(lista) > 1:
                return locate_next(' '.join(lista[1:]), next_level, level+1)
            return next_level, level
    except AttributeError:
        print("Atribute error. Possibly misconfiguration of rules:", sys.exc_info()[0])
        raise
    except:
        print("Unexpected error:", sys.exc_info()[0])
        raise


In [2]:
rules = {'tec': {'rank': '_rank?words=',
                 'filtra': '_filter_documents?afilter=',
                 'capitulo': '_document_content/',
                 '_message': 'Informe o comando: '
                },
         'cc': '_lacre/container/',
         'll': '_lacre/lacre/',
         'log': '_lacre/log',
         'report': '_lacre/add/report'
        }
params = {'report:1': [{'name': 'conteiner', 'required': True},
                       {'name': 'status', 'required': True},
                       {'name': 'test', 'required': False}
                      ]
         }

In [3]:
print(locate_next('tec', rules))

({'rank': '_rank?words=', 'capitulo': '_document_content/', 'filtra': '_filter_documents?afilter=', '_message': 'Informe o comando: '}, 1)


In [4]:
print(locate_next('tec filtra', rules))

('_filter_documents?afilter=', 2)


In [5]:
print(locate_next('tec fake', rules))

({}, 2)


In [6]:
def process_parameters(name, level, params):
    key = name.strip() + ':' + str(level)
    param_list = params.get(key, None)
    if param_list is None:
        return None
    result = []
    n_required = 0
    for param in param_list:
        result.append(param['name'])
        if param['required'] == True:
            n_required+=1
            
    return result, n_required        
print(process_parameters('report', 1, params))

(['conteiner', 'status', 'test'], 2)


In [7]:
def access_api_rules(text, rules, params_dict=None):
    '''Acess a JSON 'REST' API mapeada em rules
    text: a phrase, a sequence of words by space passed by the Pattern object
    rules: a dict on format rules = {'command1': {'subcommand1': 'url1',
                                                 {'subcommand2': 'url2'} },
                                     'command2': 'url3'
                                    }
    params_dict: a dict/list on format
    params = {'command:level': [{'name': 'name1', 'required': True},
                                {'name': 'name2', 'required': False}
                               ],
              'command2:level': [{'name': 'name2.1', 'required': True}]
             }
    '''
    lista = text.split(' ')
    url, level = locate_next(text, rules)
    if isinstance(url, dict):
        if url == {}:
            return 'Comando não reconhecido, saindo...', False
        message = url.pop('_message', '')
        return message + ' - '.join([key for key in url]), True
    resource = url
    str_params = ''
    if params_dict is None:
        str_params = '%20'.join(lista[level:])
        if not str_params:
            return 'Informe os parâmetros do filtro: ', True
    else:
        n_params_passed = len(lista) - level
        params_list, n_required = process_parameters(lista[level-1], level,params_dict)
        if n_params_passed < n_required:
            return 'Parâmetros obrigatórios: ' + str(n_required) + \
                   ' Ordem: ' + ' '.join(params_list) + \
                   ' Foram passados apenas: ' + str(n_params_passed)
        # else n_params_passed >= n_required
        str_params = '?'
        cont = 0
        for param in lista[level:]:
            str_params =  str_params + params_list[cont] + '=' + \
                          param + '&'
            cont += 1
        str_params = str_params[:-1] # Take off last '&'
        
    str_resposta = url + str_params
    return str_resposta, False


In [8]:
print(access_api_rules('report MSKU1234567', rules, params))

Parâmetros obrigatórios: 2 Ordem: conteiner status test Foram passados apenas: 1


In [9]:
print(access_api_rules('report MSKU1234567 TESTE', rules, params))

('_lacre/add/report?conteiner=MSKU1234567&status=TESTE', False)


In [10]:
print(access_api_rules('tec', rules))

('Informe o comando: rank - capitulo - filtra', True)


In [11]:
print(access_api_rules('tec rank', rules))

('Informe os parâmetros do filtro: ', True)


In [12]:
print(access_api_rules('tec rank paracetamol', rules))

('_rank?words=paracetamol', False)


In [13]:
print(access_api_rules('tec fake', rules))

('Comando não reconhecido, saindo...', False)


In [14]:
print(access_api_rules('cc MSKU1234567', rules))

('_lacre/container/MSKU1234567', False)
